### LangChain ChatOpenAI + PydanticOutputParser 

#poetry add pydantic


In [3]:
from dotenv import load_dotenv
import os
# .env 파일을 불러와서 환경 변수로 설정
load_dotenv(dotenv_path='.env')

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
print(OPENAI_API_KEY[:7])

gsk_yyC


In [ ]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain.output_parsers import PydanticOutputParser

from pydantic import BaseModel, Field
from typing import List
from pprint import pprint

# 출력 구조를 정의하는 Pydantic 모델
class MovieRecommendation(BaseModel):
    movie_title: str = Field(description="추천 영화 제목")
    reason: str = Field(description="추천 이유")
    genre: List[str] = Field(description="영화 장르")
    estimated_rating: float = Field(description="10점 만점에서 예상 평점")

parser = PydanticOutputParser(pydantic_object=MovieRecommendation)
# pydantic_object는 파서가 어떤 Pydantic 모델을 기준으로 파싱할지 지정하는 속성

# print(parser.get_format_instructions())

template = """
    다음 사용자 요청에 따라 영화를 추천해주세요.
    요청: {query}

    {format_instructions}    
"""

prompt = ChatPromptTemplate.from_template(template)

# 파서의 지시사항을 프롬프트에 주입
#  모델에게 이 구조를 따르라고 지시하는 프롬프트용 문자열 생성
prompt = prompt.partial(
    format_instructions=parser.get_format_instructions()
)

# ChatOpenAI 모델 초기화
#model = ChatOpenAI(temperature=0.7, model="gpt-3.5-turbo")
model = ChatOpenAI(
    #api_key=OPENAI_API_KEY,
    base_url="https://api.groq.com/openai/v1",  # Groq API 엔드포인트
    #model="meta-llama/llama-4-scout-17b-16e-instruct",
    model="moonshotai/kimi-k2-instruct-0905",
    temperature=0.7
)

# 체인 구성 및 실행
query = "1990년대 클래식한 느낌의 공포 영화 추천해줘"
chain = prompt | model | parser
output = chain.invoke({"query": query})

print(type(output))
pprint(output)


<class '__main__.MovieRecommendation'>
MovieRecommendation(movie_title='Scream (1996)', reason='1990년대 특유의 복고적이면서도 세련된 스타일을 유지한 채, 기발한 메타적 요소로 공포 장르의 전형을 재해석한 클래식', genre=['공포', '스릴러', '메타호러'], estimated_rating=8.5)


In [9]:

# 결과 출력
print(f"추천 영화: {output.movie_title}")
print(f"추천 이유: {output.reason}")
print(f"장르: {', '.join(output.genre)}")
print(f"예상 평점: {output.estimated_rating}/10")

추천 영화: Scream (1996)
추천 이유: 1990년대 특유의 복고적이면서도 세련된 스타일을 유지한 채, 기발한 메타적 요소로 공포 장르의 전형을 재해석한 클래식
장르: 공포, 스릴러, 메타호러
예상 평점: 8.5/10
